In [1]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipelineLegacy, DDIMScheduler, AutoencoderKL
from PIL import Image

from ip_adapter import IPAdapter

2024-06-10 07:49:58.017811: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 07:49:58.533218: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/Data/hala.gamouh/env/lib/python3.12/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "models/image_encoder/"
ip_ckpt = "models/ip-adapter_sd15.bin"
device = "cuda"

In [3]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)

/Data/hala.gamouh/env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# load SD pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
# load ip-adapter
ip_model = IPAdapter(pipe, image_encoder_path, ip_ckpt, device)

In [6]:
simple_cheese_prompts = {
    "BRIE DE MELUN": [
        "image of Brie de Melun on a white plate, with a plain white background, seen from above",
        "image of Brie de Melun on a wooden plate, with a kitchen background, seen from a side view",
        "image of Brie de Melun on a marble plate, with a shop background, seen from an angled shot",
        "image of Brie de Melun on a slate board, with a cheese cave background, seen from above",
        "image of Brie de Melun on a plain white background, with a plain white background, seen from a face view",
        "image of Brie de Melun on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Brie de Melun on a wooden plate, with a plain white background, seen from above",
        "image of Brie de Melun on a marble plate, with a kitchen background, seen from a side view",
        "image of Brie de Melun on a slate board, with a shop background, seen from above",
        "image of Brie de Melun on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "CAMEMBERT": [
        "image of Camembert on a white plate, with a plain white background, seen from above",
        "image of Camembert on a wooden plate, with a kitchen background, seen from a side view",
        "image of Camembert on a marble plate, with a shop background, seen from an angled shot",
        "image of Camembert on a slate board, with a cheese cave background, seen from above",
        "image of Camembert on a plain white background, with a plain white background, seen from a face view",
        "image of Camembert on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Camembert on a wooden plate, with a plain white background, seen from above",
        "image of Camembert on a marble plate, with a kitchen background, seen from a side view",
        "image of Camembert on a slate board, with a shop background, seen from above",
        "image of Camembert on a plain white background, with a cheese cave background, seen from a side view"
    ],
    "EPOISSES": [
        "image of Epoisses on a white plate, with a plain white background, seen from above",
        "image of Epoisses on a wooden plate, with a kitchen background, seen from a side view",
        "image of Epoisses on a marble plate, with a shop background, seen from an angled shot",
        "image of Epoisses on a slate board, with a cheese cave background, seen from above",
        "image of Epoisses on a plain white background, with a plain white background, seen from a face view",
        "image of Epoisses on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Epoisses on a wooden plate, with a plain white background, seen from above",
        "image of Epoisses on a marble plate, with a kitchen background, seen from a side view",
        "image of Epoisses on a slate board, with a shop background, seen from above",
        "image of Epoisses on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "FOURME D’AMBERT": [
        "image of Fourme d'Ambert on a white plate, with a plain white background, seen from above",
        "image of Fourme d'Ambert on a wooden plate, with a kitchen background, seen from a side view",
        "image of Fourme d'Ambert on a marble plate, with a shop background, seen from an angled shot",
        "image of Fourme d'Ambert on a slate board, with a cheese cave background, seen from above",
        "image of Fourme d'Ambert on a plain white background, with a plain white background, seen from a face view",
        "image of Fourme d'Ambert on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Fourme d'Ambert on a wooden plate, with a plain white background, seen from above",
        "image of Fourme d'Ambert on a marble plate, with a kitchen background, seen from a side view",
        "image of Fourme d'Ambert on a slate board, with a shop background, seen from above",
        "image of Fourme d'Ambert on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "RACLETTE": [
        "image of Raclette on a white plate, with a plain white background, seen from above",
        "image of Raclette on a wooden plate, with a kitchen background, seen from a side view",
        "image of Raclette on a marble plate, with a shop background, seen from an angled shot",
        "image of Raclette on a slate board, with a cheese cave background, seen from above",
        "image of Raclette on a plain white background, with a plain white background, seen from a face view",
        "image of Raclette on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Raclette on a wooden plate, with a plain white background, seen from above",
        "image of Raclette on a marble plate, with a kitchen background, seen from a side view",
        "image of Raclette on a slate board, with a shop background, seen from above",
        "image of Raclette on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "MORBIER": [
        "image of Morbier on a white plate, with a plain white background, seen from above",
        "image of Morbier on a wooden plate, with a kitchen background, seen from a side view",
        "image of Morbier on a marble plate, with a shop background, seen from an angled shot",
        "image of Morbier on a slate board, with a cheese cave background, seen from above",
        "image of Morbier on a plain white background, with a plain white background, seen from a face view",
        "image of Morbier on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Morbier on a wooden plate, with a plain white background, seen from above",
        "image of Morbier on a marble plate, with a kitchen background, seen from a side view",
        "image of Morbier on a slate board, with a shop background, seen from above",
        "image of Morbier on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "SAINT-NECTAIRE": [ 
     "image of Saint-Nectaire on a white plate, with a plain white background, seen from above",
"image of Saint-Nectaire on a wooden plate, with a kitchen background, seen from a side view",
"image of Saint-Nectaire on a marble plate, with a shop background, seen from an angled shot",
"image of Saint-Nectaire on a slate board, with a cheese cave background, seen from above",
"image of Saint-Nectaire on a plain white background, with a plain white background, seen from a face view",
"image of Saint-Nectaire on a white plate, with a cheese factory background, seen from an angled shot",
"image of Saint-Nectaire on a wooden plate, with a plain white background, seen from above",
"image of Saint-Nectaire on a marble plate, with a kitchen background, seen from a side view",
"image of Saint-Nectaire on a slate board, with a shop background, seen from above",
"image of Saint-Nectaire on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"POULIGNY SAINT- PIERRE": [
"image of Pouligny Saint-Pierre on a white plate, with a plain white background, seen from above",
"image of Pouligny Saint-Pierre on a wooden plate, with a kitchen background, seen from a side view",
"image of Pouligny Saint-Pierre on a marble plate, with a shop background, seen from an angled shot",
"image of Pouligny Saint-Pierre on a slate board, with a cheese cave background, seen from above",
"image of Pouligny Saint-Pierre on a plain white background, with a plain white background, seen from a face view",
"image of Pouligny Saint-Pierre on a white plate, with a cheese factory background, seen from an angled shot",
"image of Pouligny Saint-Pierre on a wooden plate, with",
"in its packaging"
],
"ROQUEFORT": [
        "image of Roquefort on a white plate, with a plain white background, seen from above",
        "image of Roquefort on a wooden plate, with a kitchen background, seen from a side view",
        "image of Roquefort on a marble plate, with a shop background, seen from an angled shot",
        "image of Roquefort on a slate board, with a cheese cave background, seen from above",
        "image of Roquefort on a plain white background, with a plain white background, seen from a face view",
        "image of Roquefort on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Roquefort on a wooden plate, with a plain white background, seen from above",
        "image of Roquefort on a marble plate, with a kitchen background, seen from a side view",
        "image of Roquefort on a slate board, with a shop background, seen from above",
        "image of Roquefort on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "COMTÉ": [
        "image of Comté on a white plate, with a plain white background, seen from above",
        "image of Comté on a wooden plate, with a kitchen background, seen from a side view",
        "image of Comté on a marble plate, with a shop background, seen from an angled shot",
        "image of Comté on a slate board, with a cheese cave background, seen from above",
        "image of Comté on a plain white background, with a plain white background, seen from a face view",
        "image of Comté on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Comté on a wooden plate, with a plain white background, seen from above",
        "image of Comté on a marble plate, with a kitchen background, seen from a side view",
        "image of Comté on a slate board, with a shop background, seen from above",
        "image of Comté on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "CHÈVRE": [
        "image of Chèvre on a white plate, with a plain white background, seen from above",
        "image of Chèvre on a wooden plate, with a kitchen background, seen from a side view",
        "image of Chèvre on a marble plate, with a shop background, seen from an angled shot",
        "image of Chèvre on a slate board, with a cheese cave background, seen from above",
        "image of Chèvre on a plain white background, with a plain white background, seen from a face view",
        "image of Chèvre on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Chèvre on a wooden plate, with a plain white background, seen from above",
        "image of Chèvre on a marble plate, with a kitchen background, seen from a side view",
        "image of Chèvre on a slate board, with a shop background, seen from above",
        "image of Chèvre on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "PECORINO": [
    "image of Pecorino on a white plate, with a plain white background, seen from above",
    "image of Pecorino on a wooden plate, with a kitchen background, seen from a side view",
    "image of Pecorino on a marble plate, with a shop background, seen from an angled shot",
    "image of Pecorino on a slate board, with a cheese cave background, seen from above",
    "image of Pecorino on a plain white background, with a plain white background, seen from a face view",
    "image of Pecorino on a white plate, with a cheese factory background, seen from an angled shot",
    "image of Pecorino on a wooden plate, with a plain white background, seen from above",
    "image of Pecorino on a marble plate, with a kitchen background, seen from a side view",
    "image of Pecorino on a slate board, with a shop background, seen from above",
    "image of Pecorino on a plain white background, with a cheese cave background, seen from a side view",
    "in its packaging"
],
    "NEUFCHATEL": [
        "image of Neufchatel on a white plate, with a plain white background, seen from above",
        "image of Neufchatel on a wooden plate, with a kitchen background, seen from a side view",
        "image of Neufchatel on a marble plate, with a shop background, seen from an angled shot",
        "image of Neufchatel on a slate board, with a cheese cave background, seen from above",
        "image of Neufchatel on a plain white background, with a plain white background, seen from a face view",
        "image of Neufchatel on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Neufchatel on a wooden plate, with a plain white background, seen from above",
        "image of Neufchatel on a marble plate, with a kitchen background, seen from a side view",
        "image of Neufchatel on a slate board, with a shop background, seen from above",
        "image of Neufchatel on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "CHEDDAR": [
        "image of Cheddar on a white plate, with a plain white background, seen from above",
        "image of Cheddar on a wooden plate, with a kitchen background, seen from a side view",
        "image of Cheddar on a marble plate, with a shop background, seen from an angled shot",
        "image of Cheddar on a slate board, with a cheese cave background, seen from above",
        "image of Cheddar on a plain white background, with a plain white background, seen from a face view",
        "image of Cheddar on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Cheddar on a wooden plate, with a plain white background, seen from above",
        "image of Cheddar on a marble plate, with a kitchen background, seen from a side view",
        "image of Cheddar on a slate board, with a shop background, seen from above",
        "image of Cheddar on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "BÛCHETTE DE CHÈVRE": [
        "image of Buchette de Chèvre on a white plate, with a plain white background, seen from above",
        "image of Buchette de Chèvre on a wooden plate, with a kitchen background, seen from a side view",
        "image of Buchette de Chèvre on a marble plate, with a shop background, seen from an angled shot",
        "image of Buchette de Chèvre on a slate board, with a cheese cave background, seen from above",
        "image of Buchette de Chèvre on a plain white background, with a plain white background, seen from a face view",
        "image of Buchette de Chèvre on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Buchette de Chèvre on a wooden plate, with a plain white background, seen from above",
        "image of Buchette de Chèvre on a marble plate, with a kitchen background, seen from a side view",
        "image of Buchette de Chèvre on a slate board, with a shop background, seen from above",
        "image of Buchette de Chèvre on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "PARMESAN": [
        "image of Parmesan on a white plate, with a plain white background, seen from above",
        "image of Parmesan on a wooden plate, with a kitchen background, seen from a side view",
"image of Parmesan on a marble plate, with a shop background, seen from an angled shot",
"image of Parmesan on a slate board, with a cheese cave background, seen from above",
"image of Parmesan on a plain white background, with a plain white background, seen from a face view",
"image of Parmesan on a white plate, with a cheese factory background, seen from an angled shot",
"image of Parmesan on a wooden plate, with a plain white background, seen from above",
"image of Parmesan on a marble plate, with a kitchen background, seen from a side view",
"image of Parmesan on a slate board, with a shop background, seen from above",
"image of Parmesan on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"SAINT- FÉLICIEN": [
        "image of Saint-Félicien on a white plate, with a plain white background, seen from above",
        "image of Saint-Félicien on a wooden plate, with a kitchen background, seen from a side view",
        "image of Saint-Félicien on a marble plate, with a shop background, seen from an angled shot",
        "image of Saint-Félicien on a slate board, with a cheese cave background, seen from above",
        "image of Saint-Félicien on a plain white background, with a plain white background, seen from a face view",
        "image of Saint-Félicien on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Saint-Félicien on a wooden plate, with a plain white background, seen from above",
        "image of Saint-Félicien on a marble plate, with a kitchen background, seen from a side view",
        "image of Saint-Félicien on a slate board, with a shop background, seen from above",
        "image of Saint-Félicien on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
"MONT D’OR": [
        "image of Mont d'Or on a white plate, with a plain white background, seen from above",
        "image of Mont d'Or on a wooden plate, with a kitchen background, seen from a side view",
        "image of Mont d'Or on a marble plate, with a shop background, seen from an angled shot",
        "image of Mont d'Or on a slate board, with a cheese cave background, seen from above",
        "image of Mont d'Or on a plain white background, with a plain white background, seen from a face view",
        "image of Mont d'Or on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Mont d'Or on a wooden plate, with a plain white background, seen from above",
        "image of Mont d'Or on a marble plate, with a kitchen background, seen from a side view",
        "image of Mont d'Or on a slate board, with a shop background, seen from above",
        "image of Mont d'Or on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "STILTON": [
        "image of Stilton on a white plate, with a plain white background, seen from above",
        "image of Stilton on a wooden plate, with a kitchen background, seen from a side view",
        "image of Stilton on a marble plate, with a shop background, seen from an angled shot",
        "image of Stilton on a slate board, with a cheese cave background, seen from above",
        "image of Stilton on a plain white background, with a plain white background, seen from a face view",
        "image of Stilton on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Stilton on a wooden plate, with a plain white background, seen from above",
        "image of Stilton on a marble plate, with a kitchen background, seen from a side view",
        "image of Stilton on a slate board, with a shop background, seen from above",
        "image of Stilton on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    
    ],
    "SCARMOZA": [
        "image of Scamorza on a white plate, with a plain white background, seen from above",
        "image of Scamorza on a wooden plate, with a kitchen background, seen from a side view",
        "image of Scamorza on a marble plate, with a shop background, seen from an angled shot",
        "image of Scamorza on a slate board, with a cheese cave background, seen from above",
        "image of Scamorza on a plain white background, with a plain white background, seen from a face view",
        "image of Scamorza on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Scamorza on a wooden plate, with a plain white background, seen from above",
        "image of Scamorza on a marble plate, with a kitchen background, seen from a side view",
        "image of Scamorza on a slate board, with a shop background, seen from above",
        "image of Scamorza on a plain white background, with a cheese cave background, seen from a side view",
        "in its packaging"
    ],
    "CABECOU": [
        "image of Cabecou on a white plate, with a plain white background, seen from above",
        "image of Cabecou on a wooden plate, with a kitchen background, seen from a side view",
        "image of Cabecou on a marble plate, with a shop background, seen from an angled shot",
        "image of Cabecou on a slate board, with a cheese cave background, seen from above",
        "image of Cabecou on a plain white background, with a plain white background, seen from a face view",
        "image of Cabecou on a white plate, with a cheese factory background, seen from an angled shot",
        "image of Cabecou on a wooden plate, with a plain white background, seen from above",
        "image of Cabecou on a marble plate, with a kitchen background, seen from a side view",
        "image of Cabecou on aslate board, with a shop background, seen from above",
"image of Cabecou on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"BEAUFORT": [
"image of Beaufort on a white plate, with a plain white background, seen from above",
"image of Beaufort on a wooden plate, with a kitchen background, seen from a side view",
"image of Beaufort on a marble plate, with a shop background, seen from an angled shot",
"image of Beaufort on a slate board, with a cheese cave background, seen from above",
"image of Beaufort on a plain white background, with a plain white background, seen from a face view",
"image of Beaufort on a white plate, with a cheese factory background, seen from an angled shot",
"image of Beaufort on a wooden plate, with a plain white background, seen from above",
"image of Beaufort on a marble plate, with a kitchen background, seen from a side view",
"image of Beaufort on a slate board, with a shop background, seen from above",
"image of Beaufort on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"MUNSTER": [
"image of Munster on a white plate, with a plain white background, seen from above",
"image of Munster on a wooden plate, with a kitchen background, seen from a side view",
"image of Munster on a marble plate, with a shop background, seen from an angled shot",
"image of Munster on a slate board, with a cheese cave background, seen from above",
"image of Munster on a plain white background, with a plain white background, seen from a face view",
"image of Munster on a white plate, with a cheese factory background, seen from an angled shot",
"image of Munster on a wooden plate, with a plain white background, seen from above",
"image of Munster on a marble plate, with a kitchen background, seen from a side view",
"image of Munster on a slate board, with a shop background, seen from above",
"image of Munster on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"CHABICHOU": [
"image of Chabichou on a white plate, with a plain white background, seen from above",
"image of Chabichou on a wooden plate, with a kitchen background, seen from a side view",
"image of Chabichou on a marble plate, with a shop background, seen from an angled shot",
"image of Chabichou on a slate board, with a cheese cave background, seen from above",
"image of Chabichou on a plain white background, with a plain white background, seen from a face view",
"image of Chabichou on a white plate, with a cheese factory background, seen from an angled shot",
"image of Chabichou on a wooden plate, with a plain white background, seen from above",
"image of Chabichou on a marble plate, with a kitchen background, seen from a side view",
"image of Chabichou on a slate board, with a shop background, seen from above",
"image of Chabichou on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"TOMME DE VACHE": [
"image of Tomme de Vache on a white plate, with a plain white background, seen from above",
"image of Tomme de Vache on a wooden plate, with a kitchen background, seen from a side view",
"image of Tomme de Vache on a marble plate, with a shop background, seen from an angled shot",
"image of Tomme de Vache on a slate board, with a cheese cave background, seen from above",
"image of Tomme de Vache on a plain white background, with a plain white background, seen from a face view",
"image of Tomme de Vache on a white plate, with a cheese factory background, seen from an angled shot",
"image of Tomme de Vache on a wooden plate, with a plain white background, seen from above",
"image of Tomme de Vache on a marble plate, with a kitchen background, seen from a side view",
"image of Tomme de Vache on a slate board, with a shop background, seen from above",
"image of Tomme de Vache on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"REBLOCHON": [
"image of Reblochon on a white plate, with a plain white background, seen from above",
"image of Reblochon on a wooden plate, with a kitchen background, seen from a side view",
"image of Reblochon on a marble plate, with a shop background, seen from an angled shot",
"image of Reblochon on a slate board, with a cheese cave background, seen from above",
"image of Reblochon on a plain white background, with a plain white background, seen from a face view",
"image of Reblochon on a white plate, with a cheese factory background, seen from an angled shot",
"image of Reblochon on a wooden plate, with a plain white background, seen from above",
"image of Reblochon on a marble plate, with a kitchen background, seen from a side view",
"image of Reblochon on a slate board, with a shop background, seen from above",
"image of Reblochon on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"EMMENTAL": [
"image of Emmental on a white plate, with a plain white background, seen from above",
"image of Emmental on a wooden plate, with a kitchen background, seen from a side view",
"image of Emmental on a marble plate, with a shop background, seen from an angled shot",
"image of Emmental on a slate board, with a cheese cave background, seen from above",
"image of Emmental on a plain white background, with a plain white background, seen from a face view",
"image of Emmental on a white plate, with a cheese factory background, seen from an angled shot",
"image of Emmental on a wooden plate, with a plain white background, seen from above",
"image of Emmental on a marble plate, with a kitchen background, seen from a side view",
"image of Emmental on a slate board, with a shop background, seen from above",
"image of Emmental on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"FETA": [
"image of Feta on a white plate, with a plain white background, seen from above",
"image of Feta on a wooden plate, with a kitchen background, seen from a side view",
"image of Feta on a marble plate, with a shop background, seen from an angled shot",
"image of Feta on a slate board, with a cheese cave background, seen from above",
"image of Feta on a plain white background, with a plain white background, seen from a face view",
"image of Feta on a white plate, with a cheese factory background, seen from an angled shot",
"image of Feta on a wooden plate, with a plain white background, seen from above",
"image of Feta on a marble plate, with a kitchen background, seen from a side view",
"image of Feta on a slate board, with a shop background, seen from above",
"image of Feta on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"OSSAU- IRATY": [
"image of Ossau-Iraty on a white plate, with a plain white background, seen from above",
"image of Ossau-Iraty on a wooden plate, with a kitchen background, seen from a side view",
"image of Ossau-Iraty on a marble plate, with a shop background, seen from an angled shot",
"image of Ossau-Iraty on a slate board, with a cheese cave background, seen from above",
"image of Ossau-Iraty on a plain white background, with a plain white background, seen from a face view",
"image of Ossau-Iraty on a white plate, with a cheese factory background, seen from an angled shot",
"image of Ossau-Iraty on a wooden plate, with a plain white background, seen from above",
"image of Ossau-Iraty on a marble plate, with a kitchen background, seen from a side view",
"image of Ossau-Iraty on a slate board, with a shop background, seen from above",
"image of Ossau-Iraty on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"MIMOLETTE": [
"image of Mimolette on a white plate, with a plain white background, seen from above",
"image of Mimolette on a wooden plate, with a kitchen background, seen from a side view",
"image of Mimolette on a marble plate, with a shop background, seen from an angled shot",
"image of Mimolette on a slate board, with a cheese cave background, seen from above",
"image of Mimolette on a plain white background, with a plain white background, seen from a face view",
"image of Mimolette on a white plate, with a cheese factory background, seen from an angled shot",
"image of Mimolette on a wooden plate, with a plain white background, seen from above",
"image of Mimolette on a marble plate, with a kitchen background, seen from a side view",
"image of Mimolette on a slate board, with ashop background, seen from above",
"image of Mimolette on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"MAROILLES": [
"image of Maroilles on a white plate, with a plain white background, seen from above",
"image of Maroilles on a wooden plate, with a kitchen background, seen from a side view",
"image of Maroilles on a marble plate, with a shop background, seen from an angled shot",
"image of Maroilles on a slate board, with a cheese cave background, seen from above",
"image of Maroilles on a plain white background, with a plain white background, seen from a face view",
"image of Maroilles on a white plate, with a cheese factory background, seen from an angled shot",
"image of Maroilles on a wooden plate, with a plain white background, seen from above",
"image of Maroilles on a marble plate, with a kitchen background, seen from a side view",
"image of Maroilles on a slate board, with a shop background, seen from above",
"image of Maroilles on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"GRUYÈRE": [
"image of Gruyère on a white plate, with a plain white background, seen from above",
"image of Gruyère on a wooden plate, with a kitchen background, seen from a side view",
"image of Gruyère on a marble plate, with a shop background, seen from an angled shot",
"image of Gruyère on a slate board, with a cheese cave background, seen from above",
"image of Gruyère on a plain white background, with a plain white background, seen from a face view",
"image of Gruyère on a white plate, with a cheese factory background, seen from an angled shot",
"image of Gruyère on a wooden plate, with a plain white background, seen from above",
"image of Gruyère on a marble plate, with a kitchen background, seen from a side view",
"image of Gruyère on a slate board, with a shop background, seen from above",
"image of Gruyère on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"MOTHAIS": [
"image of Mothais on a white plate, with a plain white background, seen from above",
"image of Mothais on a wooden plate, with a kitchen background, seen from a side view",
"image of Mothais on a marble plate, with a shop background, seen from an angled shot",
"image of Mothais on a slate board, with a cheese cave background, seen from above",
"image of Mothais on a plain white background, with a plain white background, seen from a face view",
"image of Mothais on a white plate, with a cheese factory background, seen from an angled shot",
"image of Mothais on a wooden plate, with a plain white background, seen from above",
"image of Mothais on a marble plate, with a kitchen background, seen from a side view",
"image of Mothais on a slate board, with a shop background, seen from above",
"image of Mothais on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"VACHERIN": [
"image of Vacherin on a white plate, with a plain white background, seen from above",
"image of Vacherin on a wooden plate, with a kitchen background, seen from a side view",
"image of Vacherin on a marble plate, with a shop background, seen from an angled shot",
"image of Vacherin on a slate board, with a cheese cave background, seen from above",
"image of Vacherin on a plain white background, with a plain white background, seen from a face view",
"image of Vacherin on a white plate, with a cheese factory background, seen from an angled shot",
"image of Vacherin on a wooden plate, with a plain white background, seen from above",
"image of Vacherin on a marble plate, with a kitchen background, seen from a side view",
"image of Vacherin on a slate board, with a shop background, seen from above",
"image of Vacherin on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"MOZZARELLA": [
"image of Mozzarella on a white plate, with a plain white background, seen from above",
"image of Mozzarella on a wooden plate, with a kitchen background, seen from a side view",
"image of Mozzarella on a marble plate, with a shop background, seen from an angled shot",
"image of Mozzarella on a slate board, with a cheese cave background, seen from above",
"image of Mozzarella on a plain white background, with a plain white background, seen from a face view",
"image of Mozzarella on a white plate, with a cheese factory background, seen from an angled shot",
"image of Mozzarella on a wooden plate, with a plain white background, seen from above",
"image of Mozzarella on a marble plate, with a kitchen background, seen from a side view",
"image of Mozzarella on a slate board, with a shop background, seen from above",
"image of Mozzarella on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"TÊTE DE MOINES": [
"image of Tête de Moine on a white plate, with a plain white background, seen from above",
"image of Tête de Moine on a wooden plate, with a kitchen background, seen from a side view",
"image of Tête de Moine on a marble plate, with a shop background, seen from an angled shot",
"image of Tête de Moine on a slate board, with a cheese cave background, seen from above",
"image of Tête de Moine on a plain white background, with a plain white background, seen from a face view",
"image of Tête de Moine on a white plate, with a cheese factory background, seen from an angled shot",
"image of Tête de Moine on a wooden plate, with a plain white background, seen from above",
"image of Tête de Moine on a marble plate, with a kitchen background, seen from a side view",
"image of Tête de Moine on a slate board, with a shop background, seen from above",
"image of Tête de Moine on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
],
"FROMAGE FRAIS": [
"image of Fromage Frais on a white plate, with a plain white background, seen from above",
"image of Fromage Frais on a wooden plate, with a kitchen background, seen from a side view",
"image of Fromage Frais on a marble plate, with a shop background, seen from an angled shot",
"image of Fromage Frais on a slate board, with a cheese cave background, seen from above",
"image of Fromage Frais on a plain white background, with a plain white background, seen from a face view",
"image of Fromage Frais on a white plate, with a cheese factory background, seen from an angled shot",
"image of Fromage Frais on a wooden plate, with a plain white background, seen from above",
"image of Fromage Frais on a marble plate, with a kitchen background, seen from a side view",
"image of Fromage Frais on a slate board, with a shop background, seen from above",
"image of Fromage Frais on a plain white background, with a cheese cave background, seen from a side view",
"in its packaging"
]
}







In [9]:
import os
import random
from PIL import Image

base_dir = "/Data/hala.gamouh/cheese_classification_challenge/dataset/val"
output_base_dir = "/Data/hala.gamouh/ip_adapt_on_val_only"

os.makedirs(output_base_dir, exist_ok=True)

# Function to generate images for a given cheese name and prompt
def generate_images_for_cheese(cheese_name, image_path, cheese_prompt, scale, ip_model, num_samples=1):
    image = Image.open(image_path)
    prompt = f"best quality, high quality, {cheese_prompt}"
    print(f"Generating images for {cheese_name} with prompt: {prompt}")
    
    cheese_name = cheese_name.strip().upper()
    special_cheeses = ["FOURME D’AMBERT", "ROQUEFORT", "STILTON"]
    
    if cheese_name in special_cheeses:
        images = ip_model.generate(
            pil_image=image, 
            num_samples=num_samples, 
            num_inference_steps=20, 
            seed=42, 
            prompt=prompt, 
            negative_prompt="blue stains, too blue, green stains, too green", 
            scale=scale
        )
    else:
        images = ip_model.generate(
            pil_image=image, 
            num_samples=num_samples, 
            num_inference_steps=20, 
            seed=42, 
            prompt=prompt, 
            scale=scale
        )

    return images

scales = [0.5,0.7,0.9]
#seen = ["BRIE DE MELUN", "CAMEMBERT", "EPOISSES","FOURME D’AMBERT", "MORBIER", "RACLETTE", "SAINT-NECTAIRE"]
for cheese_name in simple_cheese_prompts.keys():
    cheese_dir = os.path.join(base_dir, cheese_name)
    cheese_output_dir = os.path.join(output_base_dir, cheese_name)

    os.makedirs(cheese_output_dir, exist_ok=True)
    for scale in scales:
        for image_name in os.listdir(cheese_dir):
            image_path = os.path.join(cheese_dir, image_name)
            print(image_path)
            for _ in range(3):  # Generate images three times per scale
                drawn_prompt = random.choice(simple_cheese_prompts[cheese_name])
                full_prompt = f"best quality, high quality, {drawn_prompt}"
    
                # Generate images for the current cheese and prompt
                cheese_images = generate_images_for_cheese(cheese_name, image_path, full_prompt, scale, num_samples=1)
                
                for i, gen_image in enumerate(cheese_images):
                    output_image_path = os.path.join(cheese_output_dir, f"{cheese_name}_{image_name}_prompt{scale}_{_+1}_{i+1}_.jpg")
                    gen_image.save(output_image_path)
                        

/Data/hala.gamouh/cheese_classification_challenge/dataset/val/FOURME D’AMBERT/000000.jpg
generating images for FOURME D’AMBERT with prompt: best quality, high quality, image of Fourme d'Ambert on a plain white background, with a cheese cave background, seen from a side view


  0%|          | 0/20 [00:00<?, ?it/s]

generating images for FOURME D’AMBERT with prompt: best quality, high quality, image of Fourme d'Ambert on a plain white background, with a plain white background, seen from a face view


  0%|          | 0/20 [00:00<?, ?it/s]

generating images for FOURME D’AMBERT with prompt: best quality, high quality, image of Fourme d'Ambert on a plain white background, with a cheese cave background, seen from a side view


...

  0%|          | 0/20 [00:00<?, ?it/s]

generating images for BEAUFORT with prompt: best quality, high quality, image of Beaufort on a white plate, with a plain white background, seen from above


  0%|          | 0/20 [00:00<?, ?it/s]

generating images for BEAUFORT with prompt: best quality, high quality, image of Beaufort on a slate board, with a cheese cave background, seen from above


  0%|          | 0/20 [00:00<?, ?it/s]